In [ ]:
import os
import random  # Holds up spork
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

CROPPED_DIR = "data/CroppedYale"
UNCROPPED_DIR = "data/yalefaces"

In [ ]:
def get_all_files(parent_dir):
    x = [i for i in os.walk(parent_dir)]
    print(len(x))
    subdirs = [i for i in x[1:]]
    paths = []
    for subdir in subdirs:
        paths += [os.path.join(subdir[0], i) for i in subdir[2]]
        
    return paths
        
cropped = get_all_files(CROPPED_DIR)
original = len(cropped)
cropped = [i for i in cropped if "bad" not in i]
print("{} \"*.bad\" files removed.".format(original - len(cropped)))

In [ ]:
def read_pgm(pgm):
    return(np.asarray(Image.open(pgm)))

def read_flat_pgm(pgm):
    return np.ravel(np.asarray(Image.open(pgm)))

In [ ]:
read_pgm(cropped[0])
image_shape = read_pgm(cropped[0]).shape
print("Image Shape: {}".format(image_shape))

In [ ]:
x = read_flat_pgm(cropped[0])

In [ ]:
x.shape

In [ ]:
y = read_flat_pgm(cropped[1])

In [ ]:
np.vstack((x,y)).shape

In [ ]:
data_matrix = np.empty((x.shape[0], len(cropped)))
for col, pgm in enumerate(cropped):
    data_matrix[:, col] = read_flat_pgm(pgm)

# Sanity Check
i = random.randint(0, len(cropped))
assert(data_matrix[:, i].all() == read_flat_pgm(cropped[i]).all())

In [ ]:
def svd_and_reconstruction(data_matrix, modes=None):
    u, s, vh = np.linalg.svd(data_matrix, full_matrices=False)
    if modes is None:
        modes = s.shape[0]
        
    print("u.shape: {}; vh.shape: {}".format(u.shape, vh.shape))
    s_diag = np.zeros((u.shape[0], vh.shape[0]))
    s_diag[:u.shape[1], :u.shape[1]] = np.diag(s)
    
    return np.matmul(np.matmul(u[:,0:modes], s_diag[0:modes, 0:modes]), vh[0:modes, :])

def reconstruction(u, s, vh, modes=None):
    if modes is None:
        modes = s.shape[0]
        
    print("u.shape: {}; vh.shape: {}".format(u.shape, vh.shape))
    s_diag = np.zeros((u.shape[0], vh.shape[0]))
    s_diag[:u.shape[1], :u.shape[1]] = np.diag(s)
    
    return np.matmul(np.matmul(u[:,0:modes], s_diag[0:modes, 0:modes]), vh[0:modes, :])

In [ ]:
one_mode = svd_and_reconstruction(data_matrix, modes=1)

In [ ]:
img = one_mode[:, 0]
img = np.reshape(img, image_shape)

In [ ]:
Image.fromarray(img).show()

In [ ]:
def show_reduced_image(reduced_dataset, index):
    img = np.reshape(reduced_dataset[:, index], image_shape)
    tmp = Image.fromarray(img)
    tmp.show()
    tmp.close()
    

In [ ]:
show_reduced_image(one_mode, 0)

In [ ]:
two_modes = svd_and_reconstruction(data_matrix, modes=2)
show_reduced_image(two_modes, 0)

In [ ]:
three_modes = svd_and_reconstruction(data_matrix, modes=3)
show_reduced_image(three_modes, 0)

In [ ]:
all_modes = svd_and_reconstruction(data_matrix, 2414)
show_reduced_image(all_modes, 1)

In [ ]:
show_reduced_image(all_modes, 123)
show_reduced_image(one_mode, 123)
show_reduced_image(two_modes, 123)
show_reduced_image(three_modes, 123)